# Train & Test 

Dans ce notebook nous allons entrainer et tester des modèles pour de la détection d'objet. Pour cela nous allons utiliser des images labellisés provenant d'une caméra de surveillance sur un axe routier.

--------------

## Installation des prérequis

In [1]:
pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu

Looking in indexes: https://download.pytorch.org/whl/nightly/cpu
Note: you may need to restart the kernel to use updated packages.


## Importation des Packages

In [4]:
import torch, torchvision
import os
import json
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2

## Récupération du Dataset

In [10]:
images_dir = 'Dataset/Images'
labels_path = 'Dataset/labels/labels.json'

### Classe custom pour le Dataset

In [6]:
class CustomDataset(Dataset):
    def __init__(self, images_dir, labels_path, transform=None):
        self.images_dir = images_dir
        with open(labels_path, 'r') as f:
            self.labels = json.load(f)
        self.transform = transform
        self.image_names = [item['link'] for item in self.labels]
        self.annotations = {item['link']: item['annotations'] for item in self.labels}

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_name = self.image_names[idx]
        img_path = os.path.join(self.images_dir, image_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        annotations = self.annotations[image_name]
        boxes = []
        labels = []
        for ann in annotations:
            label = ann['labels'][0]  # Assuming one label per annotation
            if label == "Bicycle":
                label_id = 1
            elif label == "Car":
                label_id = 2
            elif label == "Pedestrian":
                label_id = 3
            elif label == "Motorbike":
                label_id = 4
            elif label == "Truck":
                label_id = 5
            elif label == "Bus":
                label_id = 6
            elif label == "Motor scooter":
                label_id = 7
            elif label == "Scooter":
                label_id = 8
            else:
                label_id = 0  # Background or unknown

            x, y, width, height = ann['coordinates'].values()
            boxes.append([x, y, x + width, y + height])
            labels.append(label_id)

### Transformations

In [8]:
transform = v2.Compose([
    v2.Resize((128, 128)),
    v2.ToTensor(),
    v2.ConvertImageDtype(torch.float32)
])

/Users/bwwwah/miniconda3/envs/myenv/lib/python3.8/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [11]:
dataset = CustomDataset(images_dir, labels_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)